In [22]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, LLM

import os
import json
import yaml

openai_api_key = os.environ.get('OPENAI_API_KEY')
if not openai_api_key:
  raise ValueError("Please set OPENAI_API_KEY environment variable")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

from crewai import Agent, Task, Crew

# Define file paths for YAML configurations
files = {
    'lead_agents': '/Users/filip/Documents/Coding/Agents/config/lead_qualification_agents.yaml',
    'lead_tasks': '/Users/filip/Documents/Coding/Agents/config/lead_qualification_tasks.yaml',
    'email_agents': '/Users/filip/Documents/Coding/Agents/config/email_engagement_agents.yaml',
    'email_tasks': '/Users/filip/Documents/Coding/Agents/config/email_engagement_tasks.yaml'
}



In [24]:
# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']


In [25]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")


In [26]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

In [27]:
# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

In [30]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()
flow.plot()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as crewai_flow.html


In [38]:
from IPython.display import IFrame

IFrame(src='/Users/filip/Documents/Coding/Agents/crewai_flow.html', width='150%', height=600)

In [40]:
emails = await flow.kickoff_async()


🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
└── 🧠 Starting Flow...

 Flow started with ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e


🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 30cf1478-bcb5-4700-933d-f721d5d40e4e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [41]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0022


/var/folders/9_/jbzzxfmd1tg5b4m5n571kf800000gn/T/ipykernel_74888/1963678899.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,14917,13449,1152,1468,6


In [42]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0002


/var/folders/9_/jbzzxfmd1tg5b4m5n571kf800000gn/T/ipykernel_74888/99626748.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1210,976,0,234,2


In [43]:
scores = flow.state["score_crews_results"]


In [44]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))





Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,8
Professional Background,"Not specified in the provided data, further research may be needed."
Company Name,Clearbit
Industry,Data Enrichment and Business Intelligence
Company Size,30
Revenue,31.500000
Market Presence,7
Lead Score,75


In [45]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

João,   Unlock the potential of AI-driven automation at Clearbit with CrewAI’s
Multi-Agent Orchestration Platform. Imagine accelerating your data enrichment
processes and enhancing business intelligence capabilities. Schedule a quick
15-minute call to see how we can tailor our platform for your team’s needs.
Ready to dive into automation? [Book your meeting now](#) and let's explore the
transformative impact CrewAI can bring to Clearbit.    Don’t miss out—each
moment spent without automation is an opportunity lost. [Claim your spot
today](#)!
